# Tutorial de Big Data
## Tutorial 4 - Componentes principales

Componentes principales (PCA, en inglés) es una técnica de **aprendizaje no supervisado**. Es decir que nos encontramos en una situación donde tenemos información de un conjunto de variables o features ($X_1, X_2, ..., X_p$), pero no sobre una variable de resultado o outcome ($Y$). Vamos a tratar de ajustar algoritmos que nos permitan entender la relación entre nuestros datos, trabajando con su propia naturaleza (su covarianza) y sin un outcome de interés $Y$.  

Esto se diferencia del **aprendizaje supervisado**, caso en el cual los estimadores se usan para **predecir** resultados basados en datos que poseen un outcome o variable de resultado $Y$ (puede ser una etiqueta -clasificación- o un valor -regresión-). Por ejemplo, una regresión lineal o una regresión logística para el caso de clasificación.

Los algoritmos de aprendizaje no supervisado pueden ser muy útiles para casos en los que se busca **reducir la dimensionalidad**, por ejemplo cuando se busca visualizar datos de gran dimensionalidad o se busca crear un índice. PCA suele emplearse como parte del **análisis descriptivo y exploratorio de datos**.

Supongamos que tenemos $n$ observaciones y $p$ variables y queremos visualizarlas como parte de una análisis exploratorio de los datos.

Podríamos realizar gráficos de a 2 variables, pero serían muchos si $p$ es grande...
Entonces vamos a buscar una representación de los datos en menos dimensiones (2 usualmente) que capture la mayor información (varianza explicada) posible.

Las dimensiones serán combinaciones lineales de las $p$ variables, resultando en direcciónes que maximizan la varianza.

Vamos a trabajar con la librería [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

### Ejemplo 1 - Crímenes violentos por estado de Estados Unidos

In [2]:
#!pip install pandas
#!pip install matplotlib
#!pip install numpy
#!pip install scikit-learn
#!pip install seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns

Vamos a trabajar con una base de datos que provee el libro ISLP.

##### Violent Crime Rates by US State
Contiene información sobre arrestos por asaltos, asesinatos y violaciones cada 100.000 habitantes en 50 estados de Estados Unidos en 1973. También tiene información sobre el porcentaje de población viviendo en zonas urbanas.

- Murder: Murder arrests (per 100,000)
- Assault: Assault arrests (per 100,000)
- Rape: Rape arrests (per 100,000)
- UrbanPop: Percent urban population

In [3]:
import os
os.chdir("/Volumes/GoogleDrive-111652351013091046884/My Drive/Docencia/Ciencia de Datos/Tutorial 2024/Tutorial4")

In [ ]:
arrests_data = pd.read_csv("USArrests.csv")
columns_names=arrests_data.columns.tolist()
print("Columns names:")
print(columns_names)

In [ ]:
arrests_data.head()

Nos vamos a quedar sólo con las columnas numéricas y esta será nuestra matriz **x** (de dimensiones nxp, con n=50 y p=4).

In [ ]:
arrests = arrests_data[["Murder", "Assault", "UrbanPop", "Rape"]]
print(arrests.head())
arrests.shape

Podemos ver también la correlación entre las variables.

In [ ]:
correlation = arrests.corr()
plt.figure(figsize=(3,3))
sns.heatmap(correlation, vmax=1, square=True,annot=True,cmap='cubehelix')

plt.title('Correlation matrix')

Ahora calulemos la media y la desviación estándar (sd) de cada columna.

In [ ]:
print(arrests.mean())
print(arrests.std())

Vemos que la media no es cero y, más críticamente, la sd no es uno. ¿Cuál es el principal problema de buscar las componentes principales con datos que no están escalados (media = 0 y sd = 1)?

Vamos a usar el scaler de `sklearn` (la función que importamos `StandardScaler`) para llevarlas a media 0 y sd 1.

In [ ]:
# Escalamos las variables
# Inicializamos el transformador
scaler = StandardScaler(with_std=True, with_mean=True)
# Aplicamos fit_transform al DataFrame
arrests_transformed = pd.DataFrame(scaler.fit_transform(arrests), columns=arrests.columns)
print(arrests_transformed.mean()) # luego de la estandarización la media es cero
print(arrests_transformed.std()) # la desviación estandar es uno
display(arrests_transformed.head())

Ahora sí, vemos que la media es 0 y la sd 1.

Ahora que tenemos las variables escaleadas veamos de nuevo la correlación. ¿Qué debería cambiar?

In [ ]:
correlation = arrests_transformed.corr()
plt.figure(figsize=(3,3))
sns.heatmap(correlation, vmax=1, square=True,annot=True,cmap='cubehelix')

plt.title('Correlation matrix')

Nota: Por defecto, PCA() centra las variables para que tengan media cero pero no las escala

Aplicamos PCA. Estamos buscando maximizar la varianza de los predictores con la restricción de normalización.

Primero vamos a ajustar el modelo.

In [11]:
# Ajustamos el modelo
pca = PCA()
arrests_pca = pca.fit_transform(arrests_transformed)


Luego podemos ver la varianza explicada por cada componente, esto está en `explained_variance_ratio_` del objeto en el que hayamos ajustados los componentes principales. En nuestro caso `pca`.

In [ ]:
# % de la Varianza explicada por los componentes 
print("Varianza explicada:", pca.explained_variance_ratio_)

También podemos ver los _loadings_. Recordemos que estos son vectores que nos dicen cómo se proyecta cada valor de las coordenadas originales en cada una de las componentes principales.

In [ ]:
# Loadings vectors
loading_vectors = pca.components_ # cada fila corresponde a un CP y cada columna, a una variable
print("Loadings:\n", pca.components_)
print("Loadings del CP1:\n",pca.components_[0]) 
pca.components_[0,0] #loadings del CP1 variable 1

Resulta interesante notar que si tomamos los *loadings* del primer componente principal y los sumamos elevados al cuadrado, es decir, calculamos su norma euclidea (en criollo, el módulo):


In [ ]:
(loading_vectors[0,0])**2+(loading_vectors[0,1])**2+(loading_vectors[0,2])**2+(loading_vectors[0,3])**2 

Esta suma es igual a 1 ya que es la restricción que pusimos cuando desarrollamos el métodos.

Podemos ver las coordenadas originales.

In [ ]:
# Coordenadas originales
arrests_transformed[0:4]

Y también extraer los *scores*. Recordemos que los *scores* son las proyecciones de los puntos originales en este nuevo espacio de coordenadas.

In [ ]:
# Scores
scores = arrests_pca # Salen directamente del elemento arrests_pca
scores[0:5] # Vemos los primeros 5

Por ejemplo, si quisiéramos el valor de la proyección en la primera componente principal del primer ítem (o sea, el primer elemento de su vector de *scores*), debemos multiplicar cada componente original por su loading y sumar todo. De forma general:

$$z_{j,m} = \phi_{j,1} x_{j,1} + \phi_{j,2} x_{j,2} + ... + \phi_{j,p} x_{j,p},$$

donde $z_{j,m}$ es la proyección del elemento j-ésimo en la componente principal m-ésima (*scores*), $\phi_{j,p}$ es el elemento  *loading*

Por ejemplo, si quisiéramos la proyección del primer elemento de la muestra en la componente principal 1 la cuenta que tendríamos que hacer es esta:

$$z_{1,1} = \phi_{1,1} x_{1,1} + \phi_{1,2} x_{1,2} + \phi_{1,3} x_{1,3} + + \phi_{1,4} x_{1,4},$$

calculémosma y comparémosla con el primer elemento de `scores[0]` (los *scores* del primer elemento):

In [ ]:
# Transformamos arrests_transformed (el escaleado) a un numpy array
arrests_array = arrests_transformed.to_numpy()

# Ahora hacemos la suma de cada componente de la primera fila multiplicado con cada componente del primer loading
print(arrests_array[0][0] * loading_vectors[0][0] + 
 arrests_array[0][1] * loading_vectors[0][1] + 
 arrests_array[0][2] * loading_vectors[0][2] +
 arrests_array[0][3] * loading_vectors[0][3])

print(scores[0])

¿Y si quisiéramos calcular la la proyección (score) del primer item en la segunda compomente principal?

In [18]:
# Queda como ejercicio

#### ---- Disgresión optativa de álgebra lineal ----
Ahora una para los fanáticos del álgebra lineal: ¿Reconocen qué operación estamos haciendo?

¡Sí, Un producto escalar entre las componentes originales y las *loadings*!

Exactamente sería que:

$$z_{1,1} = x_{1} \cdot \phi_{1}^T,$$

Hagamos esa cuenta de nuevo par ala primera componente del primer elemento de la muestra:

In [ ]:
# La función dot del módulo numpy es el producto escalar (porque en inglés se llama dot product)
# Y hacemos el producto escalar entre la primera fila de arrests_array y de loading_vectors
np.dot(arrests_array[0], loading_vectors[0].transpose())

# ¿Y la segunda?
# Queda como ejercicio

Entonces $z_{1,1} = x_{1} \cdot \phi_{1}^T$, $z_{1,2} = x_{1} \cdot \phi_{2}^T$, etc...

Por último, debe haber más de un fanático *top tier* del álgebra lineal que ya habrá notado algo.

Claro, lo que todos están pensando, que las proyecciones en las nuevas coordenadas de la primera muestra (*scores*) es el producto entre la matriz de *loadings* con el vector de la primera muestra ;). O sea:

$$z_{1} = x_{1} \times \phi^T$$


In [ ]:
# matmul es la función de numpy para la multiplicación de matrices
# Ahora lo que hacemos es multiplicar la matriz de loadings por el primer punto
np.matmul(arrests_array[0], loading_vectors.transpose())

Entonces obtenemos la primera fila de la matriz de *scores*, pero momento que hay más. 

Si $z_{1} = x_{1} \times \phi^T$, entonces la lógica indica que:

$$Z = X \times \phi^T,$$

es decir, la matriz de *scores* es el producto de matrices entre la matriz de datos y la de *loadings*.

In [ ]:
scores_a_mano = np.matmul(arrests_array, loading_vectors.transpose())
print(scores_a_mano[0:5,])
print(scores[0:5,])

#### ---- FIN disgresión optativa de álgebra lineal ----

¿Y qué pasa con la matriz de correlación de los scores?

In [ ]:
correlation = pd.DataFrame(scores).corr()
plt.figure(figsize=(5,5))
sns.heatmap(correlation, vmax=1, square=True,annot=True,cmap='cubehelix')

plt.title('Score correlation matrix')

Ahora podemos ver el biplot. Recordemos que el biplot es una representación en la que podemos ver de forma conjunta los *scores* (los puntitos) y las *loadings* (las flechitas). En general se grafica para las primeras dos componentes principales aunque a veces se puede graficar para las primeras 3.

In [ ]:
# Biplot
i, j = 0, 1 # Componentes
fig, ax = plt.subplots(1, 1, figsize=(5, 5)) # creamos 1 subplot
ax.scatter(scores[:,0], scores[:,1]) # graficamos los valores de los CP1 y CP2
ax.set_xlabel('CP%d' % (i+1))
ax.set_ylabel('CP%d' % (j+1))
for k in range(pca.components_.shape[1]): # loop que itera por la cantidad de features
    ax.arrow(0, 0, pca.components_[i,k], pca.components_[j,k]) # flecha desde el origen (0) a las coordenadas
    ax.text(pca.components_[i,k], pca.components_[j,k], arrests.columns[k]) # al final de cada flecha, nombre de la variable

Mejoremos un poquito el biplot:

In [ ]:
# Biplot
# Ajustes, extendemos longitud de las flechas e invertimos el eje y

i, j = 0, 1 # Componentes

scale_arrow = s_ = 2 # para extender la longitud de las flechas y que se vean mejor
scores[:,1] *= -1
pca.components_[1] *= -1 # gira el eje y (CP2)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(scores[:,0], scores[:,1]) 
ax.set_xlabel('CP%d' % (i+1))
ax.set_ylabel('CP%d' % (j+1))
for k in range(pca.components_.shape[1]):
    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])
    ax.text(s_*pca.components_[i,k], s_*pca.components_[j,k], arrests.columns[k])

¿Qué podemos decir sobre la relación entre las coordenadas originales y las primeras dos componentes principales?

In [ ]:
scores_state = pd.DataFrame(scores)
scores_state['State'] = arrests_data['State']
scores_state.head()

In [ ]:
# Biplot
# Ajustes, extendemos longitud de las flechas e invertimos el eje y

i, j = 0, 1 # Componentes

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(scores[:,0], scores[:,1], s=2) 
ax.set_xlabel('CP%d' % (i+1))
ax.set_ylabel('CP%d' % (j+1))
for k in range(pca.components_.shape[1]):
    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])
    ax.text(s_*pca.components_[i,k], s_*pca.components_[j,k], arrests.columns[k])

# Adding text labels
for i in range(len(scores_state)):
    ax.text(scores_state[0][i], scores_state[1][i], scores_state['State'][i], fontsize=9, ha='center', va='center', color = 'blue')


plt.show()

¿Qué podemos decir de los países?

Miremos ahora un dato importante, la proporción de varianza explicada por cada componente.

In [ ]:
# % de la Varianza explicada por los componentes 
print(pca.explained_variance_ratio_) # CP1 explica el 62% de la varianza

Esto lo podemos pensar también como la varianza de cada nueva coordenada sobre la varianza total (que es 4).

In [ ]:
print(np.var(scores[:,0])/4)
print(np.var(scores[:,1])/4)
print(np.var(scores[:,2])/4)
print(np.var(scores[:,3])/4)

Veamos esto gráficamente.

In [29]:
%%capture 
fig, axes = plt.subplots(1, 2, figsize=(10, 4)) # 2 subplots uno al lado del otro
ticks = np.arange(pca.n_components_)+1 # para crear ticks en el eje horizontal
ax = axes[0]
ax.plot(ticks, pca.explained_variance_ratio_ , marker='o')
ax.set_xlabel('Componente principal');
ax.set_ylabel('Proporción de la varianza explicada por cada componente')
ax.set_ylim([0,1])
ax.set_xticks(ticks)
# capture suprime la visualización de la figura parcialmente terminada

In [ ]:
ax = axes[1]
ax.plot(ticks, pca.explained_variance_ratio_.cumsum(), marker='o') 
ax.set_xlabel('Componente principal')
ax.set_ylabel('Suma acumulada de la varianza explicada')
ax.set_ylim([0, 1])
ax.set_xticks(ticks)
fig

¿Qué podmeos decir sobre la representación bidimensional de los datos viendo esto?

### Ejemplo 2 - Características organolépticas del vino

Otro Ejemplo

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Vamos a trabajar con un dataset de vinos (de scikit-learn). Contiene características de 178 vinos y a qué segmento de consumidores pertenecen

In [ ]:
# Importar el dataset y breve exploración
wine_data = pd.read_csv('wine.csv')
print(wine_data.shape)
print(wine_data.dtypes)
print(wine_data.head())
print(wine_data.shape)

Ahora tenemos 178 samples en un espacio de 14 coordenadas.

Estos datos los vamos a usar más adelante para aprendizaje supervisado (es decir, cuando tenemos X e Y). De momento vamos a quedar con la matriz X conformada por todas las columnas salvo `customer_segment`.

In [ ]:
# Separamos datos entre X e Y (por ahora, haremos de cuenta que no contamos con Y)
wine_features = wine_data.iloc[:, 0:13].values
wine_customer_segment = wine_data.iloc[:, 13].values

# Vemos las etiquetas posibles de customer segment
wine_customer_segment_unique, counts = np.unique(wine_customer_segment, return_counts=True)
for value, count in zip(wine_customer_segment_unique, counts):
    print(f"Value: {value}, Count: {count}")

Empecemos con las componentes principales. Primero estandarizamos:

In [34]:
# Preprocesamiento. Estandarizar las variables
# Iniciar scaler y aplicarlo
sc = StandardScaler()
wine_features_transformed = sc.fit_transform(wine_features)

¿Por qué estandarizamos? El análisis es sensible a la varianza de las variables originales y eso puede ocasionar problemas a la hora de elegir los CPs

In [ ]:
# Aplicamos PCA
pca = PCA(n_components = 2)
wine_scores = pca.fit_transform(wine_features_transformed) # Obtenemos los scores

wine_scores[0:4,]

In [ ]:
pca.components_

In [ ]:
# Como vimos más arriba, esa transformación es equivalente a el producto matricial entre los datos y los loadings
wine_scores_a_mano = np.matmul(wine_features_transformed, pca.components_.transpose())
print(wine_scores_a_mano[0:4,])

In [ ]:
# % de la Varianza explicada por los componentes
print("Varianza explicada:", pca.explained_variance_ratio_)
# El primer componente principal explica el 36% de la varianza, mientras que el segundo, explica el 19%

In [ ]:
# Loading vectors
loading_vectors = pca.components_ # cada fila corresponde a un CP y cada columna, a una variable
print("Loadings:\n", pca.components_)
print("Loadings del CP1:\n",pca.components_[0]) 

In [ ]:
# Visualizamos features y loadings
for i, loading_vector in enumerate(loading_vectors):
    print(f"\nLoading Vector CP{i+1}:")
    for j, feature in enumerate(wine_data.columns[:-1]):
        print(f"{feature}: {round(loading_vector[j],3)}")
    print()

In [ ]:
# Crear un DataFrame para los componentes principales
pca_df = pd.DataFrame(data=wine_scores, columns=['Componente_1', 'Componente_2'])

# Añadir la variable objetivo al DataFrame de los componentes principales
pca_df['Customer_Segment'] = wine_customer_segment
pca_df

In [ ]:
# Graficamos los componentes
plt.figure(figsize=(8, 6))
plt.scatter(pca_df['Componente_1'], pca_df['Componente_2'], c=wine_data['Customer_Segment'], cmap='viridis')
plt.xlabel('Principal Component 1', fontsize=11)
plt.ylabel('Principal Component 2', fontsize=11)
plt.title('PCA - Components 1 and 2')
plt.colorbar(label='Customer Segment')
plt.grid(True)
plt.show()

In [ ]:
 # Otra forma de graficar
plt.figure(figsize=(8, 6))
plt.xlabel('Principal Component 1',fontsize=12)
plt.ylabel('Principal Component 2',fontsize=12)
plt.title("Wine Data",fontsize=16)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

targets = [1, 2, 3]
colors = ['red', 'green', 'blue']

for target, color in zip(targets,colors):
    indices_graf = pca_df['Customer_Segment'] == target
    plt.scatter(pca_df.loc[indices_graf, 'Componente_1'], pca_df.loc[indices_graf, 'Componente_2'], c = color, s = 50)

#plt.xlim(-4,4)
#plt.ylim(-4,4)
plt.legend(targets)

La representación bidimensional de los datos tridimensionales capta correctamente el patrón principal de los datos: las observaciones rojas, azules y verdes, siguen estando en la representación bidimensional. 

Nota: aquí usamos dos componentes pero podríamos haber usado 1 o más de 2. Para decidir qué número de componentes usar, podemos consultar un scree plot que nos muestre la proporción de variable explicada para cada uno de los componentes y la variación en la varianza total explicada por el total de los componentes.

Típicamente se elige la cantidad de componentes para la cual la proporción de la varianza explicada cae para cada componente principal adicional (cuando hay un codo en el scree plot)

In [ ]:
# Podemos ajustar una regresión logística con los componentes
from sklearn.linear_model import LogisticRegression  
 
classifier = LogisticRegression()
classifier.fit(wine_scores, wine_customer_segment)

# Ver coeficientes
coefficients = classifier.coef_
intercept = classifier.intercept_

print("Coeficientes:")
for i, coef in enumerate(coefficients[0]):
    print(f"PC{i+1}: {coef}")


### Ejercicio - Características organolépticas del café (entre otras)

Utilice los datos de los archivos `arabica_ratings_raw.csv` y `robusta_ratings_raw.csv` para realizar un análisis de las componentes principales de las evaluaciones organolépticas del café.

Tendrá que combinar los datasets, seleccionar qué columnas son las importantes para el análisis y luego ajustar un PCA.

Es interesante que observe cómo se representa cada variedad (Arabica y Robusta) en estas componentes principales. También cuál es la varianza explicada por las mismas.

Siéntase libre de tomar las decisiones que considere necesarias (sin miedo al éxito) y concluya consecuentemente, los datos en el mundo real no vienen con instructivo.